In [ ]:
from model_manager import ModelManager
from unified_image_reader import Image
import torch
import numpy as np

manager = ModelManager('model')
checkpoint = manager.load_model(f"digpath_supervised_densenet_best_model")

In [ ]:
classes = ('Mild', 'Moderate', 'Severe')
labels = {label: idx for idx, label in enumerate(classes)}
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def diagnose_region(model, region):
    region = torch.Tensor(region[None, ::]).permute(0, 3, 1, 2).float().to(device)
    output = model(input).to(device)
    output = output.detach().squeeze().cpu().numpy()
    pred = label_decoder(np.argmax(output))
    print(f"Predicted class: {pred}")
    return pred

def label_decoder(x):
    return list(labels.keys())[list(labels.values()).index(x)]

def diagnose_wsi(model, file_path, aggregate):
    region_classifications = {}
    for i, region in enumerate(Image(file_path)):
        region = region.to(device)
        model.eval()
        pred = diagnose_region(model, region)
        region_classifications[i] = pred
    return aggregate(region_classifications)
        

def plurality_vote(region_classifications):
    return region_classifications[max(region_classifications, key=region_classifications.get)]